<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
✍🏽 תהליך הסבת שכבת גבולות תכניות לצרכי רישום לבנק"ל מודרני ✍🏽 

In [ ]:
print('4. Tazar borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  4.a Reading inputs')

In [ ]:
RawData_folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

today =  dt.date.today()
date_name =  str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name

<h1 style="text-align:right;font-size:100%">
CMS טבלת

In [ ]:
tazar_cms_cols = ['tazar_num', 'tazar_year']

tazar_cms = pd.read_excel(RawData_folder + 'Local\\cms.xlsx',
                          usecols=tazar_cms_cols)

tazar_cms.columns = ['TALAR_NUM', 'TALAR_YEAR']


del tazar_cms_cols

<h1 style="text-align:right;font-size:100%">
טבלת תצ"ר עם קידומת

In [ ]:
tazar_kidomet_cols = ['TALAR_NUM', 'TALAR_YEAR', 'KIDOMET_TALAR']

tazar_kidomet_schema = {'TALAR_NUM'    : int,
                        'TALAR_YEAR'   : int,
                        'KIDOMET_TALAR': str}

tazar_kidomet = pd.read_csv(RawData_folder + 'Local\\KIDOMET_TALAR.csv',
                            usecols    = tazar_kidomet_cols,
                            converters = tazar_kidomet_schema,
                            encoding   = 'cp1255')


del [tazar_kidomet_cols, tazar_kidomet_schema]

<h1 style="text-align:right;font-size:100%">
טבלת תצ"ר בתהליך

In [ ]:
general_inProcess_cols = ['TALAR_NUM', 'TALAR_YEAR', 'CADASTER_PROCESS', 'GUSHNUM', 'GUSHSUFFIX', 'SURVEYOR', 'TABA_NAMES']

general_inProcess_schema = {'TALAR_NUM'       : int,
                            'TALAR_YEAR'      : int,
                            'CADASTER_PROCESS': int,
                            'GUSHNUM'         : int,
                            'GUSHSUFFIX'      : int,
                            'SURVEYOR'        : int,
                            'TABA_NAMES'      : str}

general_inProcess = pd.read_excel(RawData_folder + 'Oracle\\general_inProcess.xlsx',
                                  usecols = general_inProcess_cols,
                                  converters = general_inProcess_schema)

general_inProcess.rename(columns={'GUSHNUM':'GUSH_NUM', 'GUSHSUFFIX':'GUSH_SUFFIX', 'TABA_NAMES':'PlanName'}, inplace=True)


del [general_inProcess_cols, general_inProcess_schema]

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר

In [ ]:
tazar_border_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'SURVEYOR', 'STATUS_TEXT', 'STATUS', 'TALAR_SOURCE', 'TALAR_YEAR', 'TALAR_NUM', 'TALAR_ID', 'geometry']

tazar_border_schema = {'GUSH_NUM'    : int,
                       'GUSH_SUFFIX' : int,
                       'SURVEYOR'    : int,
                       'STATUS_TEXT' : str,
                       'STATUS'      : int,
                       'TALAR_SOURCE': int,
                       'TALAR_YEAR'  : int,
                       'TALAR_NUM'   : int,
                       'TALAR_ID'    : int}

tazar_border = gpd.read_file(RawData_folder + 'CADSDE\\TALAR.gdb',
                             layer = 'TALAR',
                             include_fields = tazar_border_cols)

tazar_border = tazar_border[~tazar_border['TALAR_NUM'].isna()]


tazar_border.fillna({col:0 for col in ['GUSH_NUM','GUSH_SUFFIX','SURVEYOR','STATUS','TALAR_SOURCE','TALAR_YEAR','TALAR_NUM','TALAR_ID']}, inplace=True)

tazar_border = tazar_border.astype(tazar_border_schema)


del [tazar_border_cols, tazar_border_schema]

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר טבלאיות

In [ ]:
tazar_tabular_border_cols = ['SOURCE', 'STATUS_TEXT', 'STATUS', 'TALAR_YEAR', 'TALAR_NUM', 'TALAR_ID', 'GUSH_SUFFIX', 'GUSH_NUM', 'geometry']

tazar_tabular_border = gpd.read_file(RawData_folder + 'CADSDE\\TALAR_TABLE.gdb',
                                     layer = 'TALAR_TABLE',
                                     include_fields = tazar_tabular_border_cols)


del [tazar_tabular_border_cols]

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר בתהליך

In [ ]:
tazar_inprocess_cols = ['GUSHSUFFIX', 'GUSHNUM', 'TALAR_YEAR', 'TALAR_NUM', 'CADASTER_PROCESS', 'geometry']

tazar_inprocess = gpd.read_file(RawData_folder + 'CADSDE\\CadasterProcessBorder.gdb',
                                layer = 'CadasterProcessBorder',
                                include_fields = tazar_inprocess_cols)

tazar_inprocess.rename(columns={'GUSHNUM':'GUSH_NUM', 'GUSHSUFFIX':'GUSH_SUFFIX'}, inplace=True)

tazar_inprocess = tazar_inprocess[tazar_inprocess['CADASTER_PROCESS'] == 2]
tazar_inprocess.drop(columns=['CADASTER_PROCESS'], inplace=True)


del tazar_inprocess_cols

<h1 style="text-align:right;font-size:100%">
שכבת גבולות גושי קדסטר מודרניים

In [ ]:
Blocks_cols = ['BlockNumber', 'SubBlockNumber', 'BlockUniqueID']

Blocks = gpd.read_file(product_path,
                       layer='Blocks',
                       include_fields = Blocks_cols,
                       ignore_geometry = True)

Blocks = Blocks.astype(int)


del[Blocks_cols, RawData_folder, today, date_name, product_path]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  4.b Cleaning data')

<h1 style="text-align:right; font-size:100%">
cms הסרת רשומות ללא מספר אליפסה מטבלת

In [ ]:
tazar_cms.dropna(subset=['TALAR_NUM'], inplace=True)

<h1 style="text-align:right; font-size:100%">
סינון תהליכים שאינם תצ"ר מטבלת תהליכים כללית כאשר סוג תהליך של תצ"ר הוא 2

In [ ]:
general_inProcess = general_inProcess[general_inProcess['CADASTER_PROCESS'] == 2]

<h1 style="text-align:right; font-size:100%">
הסרת מספרי אלפיסה שאינם תקינים

In [ ]:
elipse_min = 1
elipse_max = 4000

tazar_cms = tazar_cms[tazar_cms['TALAR_NUM'].between(elipse_min, elipse_max)]
tazar_kidomet = tazar_kidomet[tazar_kidomet['TALAR_NUM'].between(elipse_min, elipse_max)]
general_inProcess = general_inProcess[general_inProcess['TALAR_NUM'].between(elipse_min, elipse_max)]
tazar_border = tazar_border[tazar_border['TALAR_NUM'].between(elipse_min, elipse_max)]
tazar_tabular_border = tazar_tabular_border[tazar_tabular_border['TALAR_NUM'].between(elipse_min, elipse_max)]
tazar_inprocess = tazar_inprocess[tazar_inprocess['TALAR_NUM'].between(elipse_min, elipse_max)]


del[elipse_min, elipse_max]

<h1 style="text-align:right; font-size:100%">
הסרת רשומות עם כפילויות (לכל מקורות המידע)

In [ ]:
tazar_cms.drop_duplicates(keep = 'last', inplace=True)
tazar_kidomet.drop_duplicates(keep = 'last', inplace=True)
general_inProcess.drop_duplicates(keep = 'last', inplace=True)
tazar_border.drop_duplicates(keep = 'last', inplace=True)
tazar_tabular_border.drop_duplicates(keep = 'last', inplace=True)
tazar_inprocess.drop_duplicates(keep = 'last', inplace=True)

<h1 style="text-align:right; font-size:100%">
טיפול בכפילויות של מספר מזהה עבור גבולות תצ"ר
<h1 style="text-align:right; font-size:100%">
טיפול בכפילויות של מספרי אליפסה מגבולות תצ"ר טבלאי (מבין הכפולים ישארו רשומות העונות למקור נתונים = 4)

In [ ]:
tazar_border.drop_duplicates(subset=['TALAR_ID'], keep='first', inplace=True)

In [ ]:
tazar_tabular_border = tazar_tabular_border.sort_values(['TALAR_NUM', 'TALAR_YEAR', 'SOURCE'])

tazar_tabular_border = tazar_tabular_border.drop_duplicates(['TALAR_NUM', 'TALAR_YEAR'], keep='first')

<h1 style="text-align:right; font-size:125%">
:איחוד מקורות טבלאיים

In [ ]:
print('  4.c Data conversion')

<h1 style="text-align:right; font-size:100%">
השלמת ערכי קידמות בטבלת תצ"ר קידומת

In [ ]:
tazar_kidomet['KIDOMET_TALAR'] = tazar_kidomet['KIDOMET_TALAR'].replace('', 'X')

<h1 style="text-align:right; font-size:100%">
הוספת שדה מקשר של מספר אליפסה מלא לצורך מיזוג

In [ ]:
for df in [tazar_cms, tazar_kidomet, general_inProcess, tazar_border, tazar_tabular_border, tazar_inprocess]:
    df[['TALAR_NUM', 'TALAR_YEAR']] = df[['TALAR_NUM', 'TALAR_YEAR']].astype('Int64')

In [ ]:
tazar_cms['ProcessName'] = tazar_cms['TALAR_NUM'].astype('str') + '/' + tazar_cms['TALAR_YEAR'].astype('str')
tazar_kidomet['ProcessName'] = tazar_kidomet['TALAR_NUM'].astype('str') + '/' + tazar_kidomet['TALAR_YEAR'].astype('str') + tazar_kidomet['KIDOMET_TALAR']
general_inProcess['ProcessName'] = general_inProcess['TALAR_NUM'].astype('str') + '/' + general_inProcess['TALAR_YEAR'].astype('str')
tazar_border['ProcessName'] = tazar_border['TALAR_NUM'].astype('str') + '/' + tazar_border['TALAR_YEAR'].astype('str')
tazar_tabular_border['ProcessName'] = tazar_tabular_border['TALAR_NUM'].astype('str') + '/' + tazar_tabular_border['TALAR_YEAR'].astype('str')
tazar_inprocess['ProcessName'] = tazar_inprocess['TALAR_NUM'].astype('str') + '/' + tazar_inprocess['TALAR_YEAR'].astype('str')

<h1 style="text-align:right; font-size:100%">
האיחוד יוצר שכבה אחת גדולה של כל רשומות הטבלאות והשכבות הקיימות

In [ ]:
tablesMerge = tazar_border.append(general_inProcess, ignore_index=True)
tablesMerge = tablesMerge.append(tazar_cms, ignore_index=True)
tablesMerge = tablesMerge.append(tazar_kidomet, ignore_index=True)
tablesMerge = tablesMerge.append(tazar_tabular_border, ignore_index=True)
tablesMerge = tablesMerge.append(tazar_inprocess, ignore_index=True)

tablesMerge.sort_values(['TALAR_NUM', 'TALAR_YEAR', 'STATUS'], inplace=True)

tablesMerge = gpd.GeoDataFrame(tablesMerge, crs=2039, geometry='geometry')

<h1 style="text-align:right; font-size:125%">
:קיבוץ טבלה
<h1 style="text-align:right; font-size:100%">
הקיבוץ יאחד כל תצ"ר לפי מספר האליפסה וישמור לכל תצ"ר את הערך הנכון בשדות

In [ ]:
dissolve_agg = {'GUSH_SUFFIX'      : 'max',
                'GUSH_NUM'         : 'max',
                'SURVEYOR'         : 'max',
                'STATUS_TEXT'      : 'last',
                'STATUS'           : 'max',
                'TALAR_SOURCE'     : 'max',
                'TALAR_YEAR'       : 'max',
                'TALAR_NUM'        : 'max',
                'TALAR_ID'         : 'max',
                'CADASTER_PROCESS' : 'max',
                'PlanName'         : 'first',
                'SOURCE'           : 'max',
                'TALAR_ID'         : 'max'}

tables_group = tablesMerge.dissolve(by = 'ProcessName',
                                    as_index = False,
                                    dropna = False,
                                    aggfunc = dissolve_agg)

tables_group = tables_group.sort_values(by = 'TALAR_ID', na_position = 'last', ignore_index= True)
tables_group.reset_index(inplace = True, drop = True)
tables_group.drop(columns = ['TALAR_NUM', 'CADASTER_PROCESS'], inplace = True)

del [dissolve_agg, tablesMerge, tazar_border, general_inProcess, tazar_cms, tazar_kidomet, tazar_tabular_border, tazar_inprocess]

<h1 style="text-align:right; font-size:100%">
מילוי ערכי מספר מזהה למקרים בהם אין מספר מזהה

In [ ]:
tables_group['TALAR_ID'] = tables_group['TALAR_ID'].astype('Int64')

last_id = tables_group['TALAR_ID'].max() + 1
length = len(tables_group[tables_group['TALAR_ID'].isna()])

na_ids = tables_group[tables_group['TALAR_ID'].isna()][['ProcessName', 'TALAR_ID']]
na_ids['TALAR_ID'] = np.arange(last_id, last_id + length).tolist()
na_ids['TALAR_ID'] = na_ids['TALAR_ID'].astype('Int64')

tables_group = tables_group.merge(na_ids, on = 'ProcessName', how = 'left', validate = 'one_to_one')
tables_group['TALAR_ID'] = tables_group['TALAR_ID_x'].fillna(tables_group['TALAR_ID_y'])
tables_group.drop(columns = ['TALAR_ID_x', 'TALAR_ID_y'], inplace = True)

<h1 style="text-align:right; font-size:100%">
מילוי ערכי מקור ריקים על ידי שדה מקור משני או 0

In [ ]:
tables_group['TALAR_SOURCE'].fillna(tables_group['SOURCE'], inplace=True)
tables_group.drop(columns = 'SOURCE', inplace=True)

<h1 style="text-align:right; font-size:100%">
ניקוי ערכי שם תב"ע ארוכים מדי לערך ריק

In [ ]:
tables_group['PlanName_len'] = tables_group['PlanName'].str.len()
tables_group.loc[tables_group['PlanName_len'] >= 35, ['PlanName']] = None
tables_group.drop(columns = 'PlanName_len', inplace=True)

<h1 style="text-align:right; font-size:100%">
יצירת שדה מקשר לטובת מיזוג עם שכבת גושים והשגת מזהה גוש בהמשך

In [ ]:
tables_group[['GUSH_NUM', 'GUSH_SUFFIX']] = tables_group[['GUSH_NUM', 'GUSH_SUFFIX']].astype('Int64')
tables_group['BlockName'] = tables_group['GUSH_NUM'].astype(str) + '_' + tables_group['GUSH_SUFFIX'].astype(str)

Blocks['BlockName'] = Blocks['BlockNumber'].astype(str) + '_' + Blocks['SubBlockNumber'].astype(str)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  4.d Defining new Layer')

<h1 style="text-align:right; font-size:100%">
איכלוס שדות

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                        #("BlockUniqueID", int), יאוחד בהמשך התהליך
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),
                        ("Status", int),
                        ("PlanName", str),
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])

tazar_new = gpd.GeoDataFrame(np.empty(0, dtype = new_columns),
                             geometry = 'geometry',
                             crs = 2039)

In [ ]:
tazar_new['geometry'] = tables_group['geometry']


tazar_new['CPBUniqueID'] = tables_group['TALAR_ID'].astype(int) + 1000000
tazar_new['CPBUniqueID'] = tazar_new['CPBUniqueID'].astype('Int64')

tazar_new['MapYear'] = tables_group['TALAR_YEAR'].astype('Int64')


tazar_new['ProcessName'] = tables_group['ProcessName'].astype('str')


tazar_new['ProcessType'] = 1  # תכנית לצרכי רישום


sourceDict = {0 : 3,
              1 : 3,
              2 : 3,
              3 : 3,
              4 : 2,
              5 : 1,
              None : 0}
tazar_new['DataSource'] = tables_group['TALAR_SOURCE'].map(sourceDict).astype('Int64')


statusDict = {0 : 1,
              1 : 1,
              2 : 2,
              3 : 4,
              4 : 5,
              5 : 9,
              6 : 3, 
              7 : 8, 
              8 : 1, 
              9 : 1, 
              None : 1}
tazar_new['Status'] = tables_group['STATUS'].map(statusDict)


geodeticNetDict = {range(0, 1998)     : 1,      # רשת ישראל הישנה
                   range(1999, 2007)  : 2,      # רשת ישראל החדשה
                   range(2008, 99999) : 3}      # רשת ישראל התקפה
tazar_new['GeodeticNetwork'] = tazar_new['MapYear'].apply(lambda x: next((v for k, v in geodeticNetDict.items() if x in k), 0))


tazar_new['SurveyorLicenseID'] = tables_group['SURVEYOR'].fillna(0)
tazar_new['SurveyorLicenseID'] = tazar_new['SurveyorLicenseID'].astype('Int64')


tazar_new['PlanName'] = tables_group['PlanName'].astype(str)


tazar_new['BlockName'] = tables_group['BlockName']
tazar_new = tazar_new.merge(Blocks, on='BlockName', how='left')
tazar_new['BlockUniqueID'] = tazar_new['BlockUniqueID'].astype('Int64')
tazar_new.drop(columns=['BlockName', 'BlockNumber', 'SubBlockNumber'], inplace=True)

In [ ]:
tazar_new.sort_values(['MapYear', 'CPBUniqueID'], inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  4.e Exporting results to product folder')

In [ ]:
tazar_new_geom    = tazar_new[~tazar_new['geometry'].isna()]
tazar_new_no_geom = tazar_new[tazar_new['geometry'].isna()]

#hebrew encoding charaters from csv cannot be inserted to feature class
tazar_new_no_geom['PlanName'] = None     

In [ ]:
no_geom_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\tazar_no_geom.csv'
json_dir        = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\Tazar_geom.geojson'


tazar_new_no_geom.drop(columns='geometry').to_csv(no_geom_csv_dir, index=False)

tazar_new_geom.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r'M:\ncdbScripts\scripts\arcpyPro\Tazar-Save as Feature Class.bat']);